In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_parquet("top_5_delay.parquet")

array(['LAX', 'DEN', 'ORD', 'DFW', 'ATL'], dtype=object)

In [76]:
# Agrupamos los datos por aerolinea, fecha, y aeropuerto de origen y contamos el número de vuelos
data = pd.DataFrame(df[df["ARRIVAL_DELAY"]>0].groupby(["DATE","ORIGIN_AIRPORT"])["FLIGHT_NUMBER"].count()).reset_index()
#data = pd.DataFrame(df[df["ARRIVAL_DELAY"]>0].groupby(["AIRLINE","DATE","ORIGIN_AIRPORT"])["FLIGHT_NUMBER"].count()).reset_index()
data = data.rename(columns={"FLIGHT_NUMBER":"DELAYED_FLIGHTS"})
data.head()

,DATE,ORIGIN_AIRPORT,DELAYED_FLIGHTS
0,2015-01-01,ATL,162
1,2015-01-01,DEN,406
2,2015-01-01,DFW,453
3,2015-01-01,LAX,236
4,2015-01-01,ORD,271


Pueder ser que haya aeropuertos que no tengan retrasos algún día (caso raro). Para realizar el forecast necesitamos tener datos en todos los días de de todos los aeropuertos para todas las aerolíneas así que vamos a asegurarnos de que esto ocurra. Para ello crearemos un dataframe auxiliar con contenga todos los datos de fechas y aeropuertos, pero sin el número de vuelos retrasados

In [79]:
dates = [datetime(2015,1,1) + timedelta(days=d) for d in range((datetime(2015,12,31) - datetime(2015,1,1)).days + 1)]*5

airports = []
for airport in df['ORIGIN_AIRPORT'].unique():
    airport_dates=[airport]*365
    airports=airports+airport_dates

# Creamos el dataframe
aux = pd.DataFrame()
aux['DATE'] = dates
aux['ORIGIN_AIRPORT'] = airports

aux.head()

,DATE,ORIGIN_AIRPORT
0,2015-01-01,LAX
1,2015-01-02,LAX
2,2015-01-03,LAX
3,2015-01-04,LAX
4,2015-01-05,LAX


In [72]:
# dates = [datetime(2015,1,1) + timedelta(days=d) for d in range((datetime(2015,12,31) - datetime(2015,1,1)).days + 1)]*5*len(df['AIRLINE'].unique())

# airlines = []
# for airline in df['AIRLINE'].unique():
#     airline_dates=[airline]*365*5
#     airlines=airlines+airline_dates
    
# airports = []
# for airport in df['ORIGIN_AIRPORT'].unique():
#     airport_dates=[airport]*365*14
#     airports=airports+airport_dates

# # Creamos el dataframe
# aux = pd.DataFrame()
# aux['DATE'] = dates
# aux['AIRLINE'] = airlines
# aux['ORIGIN_AIRPORT'] = airports

# aux.head()

Una vez hecho esto vamos a unir el dataframe actual con el obtenido a partir del groupby, mediante un left join, de tal manera que si no tenemos datos de algún aeropuerto en alguna fecha exista una fila con valor nulo, el cual sustituiremos por cero

In [80]:
forecast_data = aux.merge(data, on=['DATE','ORIGIN_AIRPORT'], how='left')
forecast_data = forecast_data.fillna(0)
forecast_data['DELAYED_FLIGHTS'] = forecast_data['DELAYED_FLIGHTS'].astype(int)
forecast_data.head()

,DATE,ORIGIN_AIRPORT,DELAYED_FLIGHTS
0,2015-01-01,LAX,236
1,2015-01-02,LAX,409
2,2015-01-03,LAX,437
3,2015-01-04,LAX,467
4,2015-01-05,LAX,400


In [81]:
# Guardamos
forecast_data.to_parquet("forecast_data.parquet", index=False)